# Visualisation of LFP & Sleep scoring

### Load LFP and packages

Load packages

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
import ephyviewer
from scipy.stats import zscore
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json
from matplotlib import cm
from matplotlib.colors import to_hex

%matplotlib widget

In [2]:
Channels = '//10.69.168.1/crnldata/forgetting/Aurelie/allLFPChannels_perMice.xlsx'
allchannels = pd.read_excel(Channels, index_col=0)

Choose OpenEphys folder

In [3]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=Path(dpath).parent
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/forgetting/Aurelie/"


fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\forgetting\Carla\OpenEphysData\Violet\VioletAHAD02_03_2025-02-25_16-0…

Load LFPs data 

In [ ]:
folder_base = Path(dpath) 
LFPfile = Path(f'{folder_base}\continuousDS.npy')
LFPfile2 = Path(f'{folder_base}\RawDataChannelExtractedDS.npy')
LFPfile3=Path(f'{folder_base}\continuous.dat')

if LFPfile.exists():
    print('continuousDS.npy file')
    All = np.load(LFPfile, mmap_mode= 'r')
    samplerate=1000 
    numchannel=All.shape[1]
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
elif LFPfile2.exists():
    print('RawDataChannelExtractedDS.npy file')
    LFPfile = LFPfile2
    All = np.load(LFPfile, mmap_mode= 'r')
    samplerate=1000 
    numchannel=32
elif LFPfile3.exists():
    print('continuous.dat file')
    LFPfile = LFPfile3
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    if filepath.exists():    
        with open(filepath) as f:
            metadata = json.load(f)
        samplerate=metadata['continuous'][0]['sample_rate']  
        numchannel=metadata['continuous'][0]['num_channels']  
    else:
        samplerate=1000 
        numchannel=32
    All = DataRec.reshape(-1,numchannel)
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
else: 
    print('no LFPs file found')

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(All.shape[0]/samplerate/60), 'min of recording')

continuous.dat file
LFPs timestamps file =  \\10.69.168.1\crnldata\forgetting\Carla\OpenEphysData\Violet\VioletAHAD02_03_2025-02-25_16-05-46_EPSP50_ms\Record Node 101\experiment1\recording1\continuous\OE_FPGA_Acquisition_Board-100.Rhythm Data\timeStamps.npy
sample rate = 10000.0 Hz
32 channels recorded
14 min of recording


Downsample LFP data to 1kHz if needed & save it

In [ ]:
if samplerate > 1000:
    datalen = np.shape(All)[0]
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(datalen * new_sampling_rate / samplerate)
    AllDS=signal.resample(All, Nmber_points, axis = 0)
    LFPtimestampsDS=LFPtimestamps[::10][:-1]
    samplerate=new_sampling_rate
    np.save(f'{LFPfile.parent}/continuousDS.npy', AllDS)
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    All=AllDS
    LFPtimestamps=LFPtimestampsDS
# eventually delete original files to gain space

Load LFPs channels

In [5]:
mouse=[]
for mouse_name in allchannels.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)

sCh=0 
all_ch_selected=1
if len(mouse)>1: # found multiple mouse name in the path
    if numchannel==32 : #  Only one mouse recorded
        ID = 0 # change the index if this is not the good mouse name
        print(f"/!\ Mutliple mouse names in the path = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    elif numchannel % 32 == 0: # Multiple mice recorded at the same time
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        sCh= 32*ID 
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    else: # the number of channels recorded is not a multiple of 32 = you probably selected manually which channel to record 
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.\n/!\ Channels were manually selected for the recording.")
        mouse=mouse[ID] 
        all_ch_selected=0 # not all channels were recorded
elif len(mouse)==1: # found one mouse name in the path
    mouse=mouse[0]

if mouse:
    RecordedArea=[]
    ChoosenChannels=[]
    combined=[]
    num_ch=0
    for idx, channels in enumerate(allchannels.loc[mouse]):
        region=allchannels.loc[mouse].index[idx]
        if isinstance(allchannels.loc[mouse].loc[region], str): # at least 2 channels
            RecordedArea.append(region)    
            ch_list = channels.split('_')
            ChoosenChannels.append(ch_list[:2])  if all_ch_selected else ChoosenChannels.append([num_ch, num_ch+1])    
            if len(ch_list)>=2: # only take the 2 first channels (change the order in the excel file if you want another pair)
                LFP=All[:,int(ch_list[0])+sCh]-All[:,int(ch_list[1])+sCh] if all_ch_selected else All[:,num_ch]-All[:,num_ch+1]
                num_ch+=2
            locals()[region]=LFP
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
        elif isinstance(allchannels.loc[mouse].loc[region], (np.int64, int)): # only one channel
            RecordedArea.append(region)  
            ch=allchannels.loc[mouse].loc[region]  
            ChoosenChannels.append(ch) if all_ch_selected else ChoosenChannels.append(num_ch)
            LFP=All[:,ch+sCh] if all_ch_selected else All[:,num_ch]
            num_ch+=1
            locals()[region]=LFP
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
    print(mouse)
    print(RecordedArea)
    print(ChoosenChannels) 
else:
    print("/!\ No mouse name found in the path OR in the excel file 'allLFPChannels_perMice.xlsx'")
    mouse='' # fill mouse name
    RecordedArea=['EMG','S1','PFC','CA1'] 
    EMG=All[:,0]
    S1=All[:,0]-All[:,0]
    PFC=All[:,0]-All[:,0]
    CA1=All[:,0]-All[:,0]
    combined=np.stack([zscore(EMG), zscore(S1), zscore(PFC), zscore(CA1)], axis=1)

Violet
['EMG', 'S1', 'PFC', 'oPFC', 'EnthC', 'RH']
[6, ['1', '3'], 11, ['25', '27'], 31, 23]


### Score sleep manually with ephyviewer


4 stages : Wake, NREM, REM, undefined

In [ ]:
labels = ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] # Wake, NREM sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen= 20 # default window length in sec

SleepScoring_filename = f'{dpath}\Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize']= winlen
view2.params['new_epoch_step']= epoch_dur

view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69cfff' #NREM
view2.by_label_params['label2', 'color']= '#ff69cd' #REM
view2.by_label_params['label3', 'color']= '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= winlen
for idx, ch in enumerate(RecordedArea): 
    view3.by_channel_params[f'ch{idx}', 'clim'] = 1
    view3.by_channel_params[f'ch{idx}', 'visible'] = True
view3.by_channel_params['ch0', 'visible'] = False # EMG needs no FFT


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

6 stages : AW, QW, NREM, IS, REM, undefined

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen= 20 # default window length in sec

SleepScoring_filename = f'{dpath}\Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view2.params['xsize']= winlen
view2.params['new_epoch_step']= epoch_dur
view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69ffe6' #QW
view2.by_label_params['label2', 'color']= '#69cfff' #NREM
view2.by_label_params['label3', 'color']= '#cd69ff' #IS
view2.by_label_params['label4', 'color']= '#ff69cd' #REM
view2.by_label_params['label5', 'color']= '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= winlen
for idx, ch in enumerate(RecordedArea): 
    view3.by_channel_params[f'ch{idx}', 'clim'] = 1
    view3.by_channel_params[f'ch{idx}', 'visible'] = True
view3.by_channel_params['ch0', 'visible'] = False # EMG needs no FFT


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

### Check all channels

In [ ]:
app = mkQApp()
winlen= 20 # default window length in sec
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
view1 = TraceViewer.from_numpy(All, samplerate, t_start, 'Signals')

cmap = cm.Spectral
values = np.linspace(0, 1, All.shape[1])
colormap = [to_hex(rgb) for rgb in cmap(values)]
for idx in range(All.shape[1]): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx]

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.params['xsize']= winlen
win.add_view(view1)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)

#Run
win.show()
app.exec()


Run the cell bellow if you believe omnetic headstage was reverted for this mouse during this session (if you run it twice it will go back to the initial configuration)

In [ ]:
Allcorrected=All.copy()
Allcorrected[:, 0]=All[:, 16]
Allcorrected[:, 1]=All[:, 17]
Allcorrected[:, 2]=All[:, 18]
Allcorrected[:, 3]=All[:, 19]
Allcorrected[:, 4]=All[:, 20]
Allcorrected[:, 5]=All[:, 21]
Allcorrected[:, 6]=All[:, 22]
Allcorrected[:, 7]=All[:, 23]
Allcorrected[:, 8]=All[:, 24]
Allcorrected[:, 9]=All[:, 25]
Allcorrected[:, 10]=All[:, 26]
Allcorrected[:, 11]=All[:, 27]
Allcorrected[:, 12]=All[:, 28]
Allcorrected[:, 13]=All[:, 29]
Allcorrected[:, 14]=All[:, 30]
Allcorrected[:, 15]=All[:, 31]
Allcorrected[:, 16]=All[:, 0]
Allcorrected[:, 17]=All[:, 1]
Allcorrected[:, 18]=All[:, 2]
Allcorrected[:, 19]=All[:, 3]
Allcorrected[:, 20]=All[:, 4]
Allcorrected[:, 21]=All[:, 5]
Allcorrected[:, 22]=All[:, 6]
Allcorrected[:, 23]=All[:, 7]
Allcorrected[:, 24]=All[:, 8]
Allcorrected[:, 25]=All[:, 9]
Allcorrected[:, 26]=All[:, 10]
Allcorrected[:, 27]=All[:, 11]
Allcorrected[:, 28]=All[:, 12]
Allcorrected[:, 29]=All[:, 13]
Allcorrected[:, 30]=All[:, 14]
Allcorrected[:, 31]=All[:, 15]
All= Allcorrected

np.save(f'{LFPfile.parent}/continuousDS.npy', All) # save an reverted version that will be loaded in priority 


mouse=[]
for mouse_name in allchannels.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)

sCh=0 
all_ch_selected=1
if len(mouse)>1: # found multiple mouse name in the path
    if numchannel==32 : #  Only one mouse recorded
        ID = 0 # change the index if this is not the good mouse name
        print(f"/!\ Mutliple mouse names in the path = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    elif numchannel % 32 == 0: # Multiple mice recorded at the same time
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        sCh= 32*ID 
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse=mouse[ID] 
    else: # the number of channels recorded is not a multiple of 32 = you probably selected manually which channel to record 
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.\n/!\ Channels were manually selected for the recording.")
        mouse=mouse[ID] 
        all_ch_selected=0 # not all channels were recorded
elif len(mouse)==1: # found one mouse name in the path
    mouse=mouse[0]

if mouse:
    RecordedArea=[]
    ChoosenChannels=[]
    combined=[]
    num_ch=0
    for idx, channels in enumerate(allchannels.loc[mouse]):
        region=allchannels.loc[mouse].index[idx]
        if isinstance(allchannels.loc[mouse].loc[region], str): # at least 2 channels
            RecordedArea.append(region)    
            ch_list = channels.split('_')
            ChoosenChannels.append(ch_list[:2])  if all_ch_selected else ChoosenChannels.append([num_ch, num_ch+1])    
            if len(ch_list)>=2: # only take the 2 first channels (change the order in the excel file if you want another pair)
                LFP=All[:,int(ch_list[0])+sCh]-All[:,int(ch_list[1])+sCh] if all_ch_selected else All[:,num_ch]-All[:,num_ch+1]
                num_ch+=2
            locals()[region]=LFP
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
        elif isinstance(allchannels.loc[mouse].loc[region], (np.int64, int)): # only one channel
            RecordedArea.append(region)  
            ch=allchannels.loc[mouse].loc[region]  
            ChoosenChannels.append(ch) if all_ch_selected else ChoosenChannels.append(num_ch)
            LFP=All[:,ch+sCh] if all_ch_selected else All[:,num_ch]
            num_ch+=1
            locals()[region]=LFP
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
    print(mouse)
    print(RecordedArea)
    print(ChoosenChannels) 
else:
    print("/!\ No mouse name found in the path OR in the excel file 'allLFPChannels_perMice.xlsx'")
    mouse='' # fill mouse name
    RecordedArea=['EMG','S1','PFC','CA1'] 
    EMG=All[:,0]
    S1=All[:,0]-All[:,0]
    PFC=All[:,0]-All[:,0]
    CA1=All[:,0]-All[:,0]
    combined=np.stack([zscore(EMG), zscore(S1), zscore(PFC), zscore(CA1)], axis=1)